## Langchain components and the Runnable Protocol

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from  langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)
os.environ.get("MODEL_OPENAI")

'gpt-4o-mini'

In [3]:
model = ChatOpenAI(temperature=0.5)
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")


In [4]:
chain = prompt | model 

In [5]:
chain

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7b204d3fb7c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7b204d222110>, root_client=<openai.OpenAI object at 0x7b204d7d2dd0>, root_async_client=<openai.AsyncOpenAI object at 0x7b204d3fb820>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

### Langchain components and the Runnable Protocol

### Standard Interface 
1. stream : stream back chunks of response
2. invoke : invoke chain with an input
3. batch  : invoke chain with a python list of inputs

### Asynchronous 
1. astream
2. ainvoke
3. abatch
4. astream_logs

### Input Types
1. Prompt : dictionary
2. Retriever : Single string
3. LLM, ChatModel : Single string, message list or PromptValue
4. Tool : single string and dictionary(which tool you are using)
5. OutputParser : depends on the parser


### Output Types
1. LLM : String 
2. Retreiver : List of Documents
3. ChatModel : Chat Message
4. Prompt : PromptValue
5. OutputParser : depends on the parser
6. Tool : Depends on the tool



In [10]:
chain.input_schema.schema()

/tmp/ipykernel_223557/3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [11]:
chain.invoke({"topic": "Football"}) # provide key value pairs 

AIMessage(content='Why did the football coach go to the bank?\nTo get his quarterback!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 13, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BL0lUdinoCMz42CapQFuAZj5tFuAI', 'finish_reason': 'stop', 'logprobs': None}, id='run-a541f75a-6583-44ae-83d7-32c91b0cede4-0', usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
## BATCH INPUT, List of Input 
chain.batch([{"topic": "AI"}, {"topic" : "Science"}])

# we can add multiple queries like this

[AIMessage(content='Why did the robot go on a diet? Because it had too many bytes!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 13, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BL0nC8qbE1S6lizeawmbIiDdNKwiP', 'finish_reason': 'stop', 'logprobs': None}, id='run-befb3ab7-bb91-449f-bd23-edd89379cb89-0', usage_metadata={'input_tokens': 13, 'output_tokens': 17, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content="Why do chemists like nitrates so much?\n\nBecause they're cheaper than day rates!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'co

In [14]:
# for concurrency
chain.batch([{"topic": "AI"}, {"topic" : "Science"}], config={"max_concurrency" : 5})


[AIMessage(content='Why did the AI break up with its robot girlfriend? Because she had too many bugs in her system!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 13, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BL0onqKhZ7yXuWWDFJij8o6SRYI1Y', 'finish_reason': 'stop', 'logprobs': None}, id='run-ccc84ab3-85da-4953-a8bb-42baec8dcb41-0', usage_metadata={'input_tokens': 13, 'output_tokens': 22, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='Why did the physics teacher break up with the biology teacher? There was no chemistry.', additional_kwargs={'refusal': None}, 

### Async Invoke

In [16]:
async for s in chain.astream({"topic" : "Satellites"}):
    print(s.content, end="", flush=True)

Why did the satellite break up with the GPS system? Because it couldn't find space for commitment!

In [18]:
await chain.abatch([{"topic" : "AI", "topic": "Science"}])

[AIMessage(content="Why do chemists like nitrates so much?\n\nBecause they're cheaper than day rates!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BL0rk5Lp9XIAJELUFxY3k7tfmyf5b', 'finish_reason': 'stop', 'logprobs': None}, id='run-91012620-d086-4c7e-b60d-c21843210a65-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]